In [13]:
from keras.models import load_model
import cv2
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import os
from keras.utils import np_utils

In [29]:
data_path = 'dataset'
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

img_size=100
data=[]
target=[]


for category in categories:
    folder_path = os.path.join(data_path,category)
    img_names = os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])

        except Exception as e:
            print('Exception:',e)
            
            
data = np.array(data)/255.0
data = np.reshape(data,(data.shape[0],img_size,img_size,1))
target = np.array(target)

new_target = np_utils.to_categorical(target)

{'with mask': 0, 'without mask': 1}
['with mask', 'without mask']
[0, 1]


In [30]:
print(data.shape)
unique, counts = np.unique(target, return_counts=True)

print(np.asarray((unique, counts)).T)
target = new_target

(1376, 100, 100, 1)
[[  0 690]
 [  1 686]]


In [31]:
# model=Sequential()

# model.add(Conv2D(32,(3,3),padding="valid",input_shape=data.shape[1:]))
# model.add(BatchNormalization())
# model.add(Activation('relu'))


# model.add(Conv2D(64,(3,3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D(64,(3,3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D(128,(3,3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))


# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(3200,activation='relu'))
# model.add(Dense(64,activation='relu'))
# model.add(Dense(2,activation='softmax'))
# #The Final layer with two outputs for two categories

# model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 98, 98, 32)        320       
_________________________________________________________________
batch_normalization_13 (Batc (None, 98, 98, 32)        128       
_________________________________________________________________
activation_18 (Activation)   (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 96, 96, 64)        18496     
_________________________________________________________________
batch_normalization_14 (Batc (None, 96, 96, 64)        256       
_________________________________________________________________
activation_19 (Activation)   (None, 96, 96, 64)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 48, 48, 64)       

In [34]:
# # data=np.load('data.npy')
# # target=np.load('target.npy')

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
31/31 [==============================] - 36s 1s/step - loss: 0.7006 - accuracy: 0.5778 - val_loss: 0.6085 - val_accuracy: 0.6895
Epoch 2/20
31/31 [==============================] - 36s 1s/step - loss: 0.5432 - accuracy: 0.7384 - val_loss: 0.4687 - val_accuracy: 0.8065
Epoch 3/20
31/31 [==============================] - 35s 1s/step - loss: 0.3690 - accuracy: 0.8434 - val_loss: 0.4134 - val_accuracy: 0.8185
Epoch 4/20
31/31 [==============================] - 35s 1s/step - loss: 0.2397 - accuracy: 0.9182 - val_loss: 0.2859 - val_accuracy: 0.8669
Epoch 5/20
31/31 [==============================] - 35s 1s/step - loss: 0.2069 - accuracy: 0.9141 - val_loss: 0.2510 - val_accuracy: 0.8992
Epoch 6/20
31/31 [==============================] - 37s 1s/step - loss: 0.1481 - accuracy: 0.9455 - val_loss: 0.1801 - val_accuracy: 0.9355
Epoch 7/20
31/31 [==============================] - 37s 1s/step - loss: 0.1415 - accuracy: 0.9455 - val_loss: 0.1793 - val_accuracy: 0.9315
Epoch 8/20
31/31 [==

In [36]:
face_clsfr= cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


source = cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

classes = None
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

while(True):
    ret,img=source.read()
#     image = img
#     Width = image.shape[1]
#     Height = image.shape[0]
    
#     net.setInput(cv2.dnn.blobFromImage(image, 0.00392, (416,416), (0,0,0), True, crop=False))
#     layer_names = net.getLayerNames()
#     output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
#     outs = net.forward(output_layers)

#     class_ids = []
#     confidences = []
#     boxes = []

#     #create bounding box 
#     for out in outs:
#         for detection in out:
#             scores = detection[5:]
#             class_id = np.argmax(scores)
#             confidence = scores[class_id]
#             if confidence > 0.1:
#                 center_x = int(detection[0] * Width)
#                 center_y = int(detection[1] * Height)
#                 w = int(detection[2] * Width)
#                 h = int(detection[3] * Height)
#                 x = center_x - w / 2
#                 y = center_y - h / 2
#                 class_ids.append(class_id)
#                 confidences.append(float(confidence))
#                 boxes.append([x, y, w, h])


#     indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)

#     #check if is people detection
#     for i in indices:
#         i = i[0]
#         box = boxes[i]
#         if class_ids[i]==0:
#             label = str(classes[class_id]) 
#             cv2.rectangle(image, (round(box[0]),round(box[1])), (round(box[0]+box[2]),round(box[1]+box[3])), (0, 0, 0), 2)
#             cv2.putText(image, label, (round(box[0])-10,round(box[1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
    
    

    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.1,4)  

    for (x,y,w,h) in faces:
#         print('inside loop')
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
#         label=0
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()